In [100]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder \
  .appName('clean_external_products') \
  .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar') \
  .getOrCreate()

In [101]:
table = "becade_rgarciaf.stg_external_procuts"
external_products = spark.read \
  .format("bigquery") \
  .option("table", table) \
  .load()

external_products.printSchema()

root
 |-- app_sale_price: double (nullable = true)
 |-- app_sale_price_currency: string (nullable = true)
 |-- country: string (nullable = true)
 |-- evaluate_rate: string (nullable = true)
 |-- isbestseller: boolean (nullable = true)
 |-- isprime: boolean (nullable = true)
 |-- original_price: string (nullable = true)
 |-- product_detail_url: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_main_image_url: string (nullable = true)
 |-- product_title: string (nullable = true)



In [103]:
import pandas as pd

In [104]:
external_products_2 = external_products.toPandas()

In [105]:
def clean_currency(data):
    for i in data.index:
            if((data["app_sale_price_currency"][i]=='' or data["app_sale_price_currency"][i]==None) and (data["country"][i]=="US" or data["country"][i]=="AU" or data["country"][i]=="CA")):
                data["app_sale_price_currency"][i]="$"

            if( (data["app_sale_price_currency"][i]=='' or data["app_sale_price_currency"][i]==None) and data["country"][i]=="BR"):
                data["app_sale_price_currency"][i]="R$"

            if((data["app_sale_price_currency"][i]=='' or data["app_sale_price_currency"][i]==None) and (data["country"][i]=="FR" or data["country"][i]=="DE" or data["country"][i]=="IT" or data["country"][i]=="NL" or data["country"][i]=="ES")):
                data["app_sale_price_currency"][i]="€"

            if((data["app_sale_price_currency"][i]=='' or data["app_sale_price_currency"][i]==None) and data["country"][i]=="IN"):
                data["app_sale_price_currency"][i]="₹"

            if((data["app_sale_price_currency"][i]=='' or data["app_sale_price_currency"][i]==None) and data["country"][i]=="TR"):
                data["app_sale_price_currency"][i]="TL"

            if((data["app_sale_price_currency"][i]=='' or data["app_sale_price_currency"][i]==None) and data["country"][i]=="AE"):
                data["app_sale_price_currency"][i]="AED"

            if((data["app_sale_price_currency"][i]=='' or data["app_sale_price_currency"][i]==None) and data["country"][i]=="GB"):
                data["app_sale_price_currency"][i]="£"

            if((data["app_sale_price_currency"][i]=='' or data["app_sale_price_currency"][i]==None) and data["country"][i]=="JP"):
                data["app_sale_price_currency"][i]="¥"

            if((data["app_sale_price_currency"][i]=='' or data["app_sale_price_currency"][i]==None) and data["country"][i]=="SG"):
                data["app_sale_price_currency"][i]="S$"

    print('Added missing currencies')
    return data

In [106]:
external_products_2 = clean_currency(external_products_2)

/opt/conda/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Added missing currencies


In [107]:
table_tasas = "becade_rgarciaf.stg_tasas_cambio_pais_anual "
tasas = spark.read \
  .format("bigquery") \
  .option("table", table_tasas) \
  .load()

In [108]:
tasas_pd = tasas.toPandas()

In [109]:
def generate_app_price_us(data,tasas):
    products_standard_price = {'isbestseller': [] , 'product_id': [] ,'app_sale_price': [], 'app_sale_price_currency': [],'isprime' : [],'evaluate_rate':[],'country':[],'app_sale_price_us':[]}  
    data_sale_price_us=0
    for i in data.index:
        if(data['country'][i]=="US"):
            data_sale_price_us= (float(data['app_sale_price'][i])*1)

        if(data['country'][i]=="AU"):
            data_sale_price_us= (float(data['app_sale_price'][i])/float(tasas_pd.loc[((tasas_pd['currency_year'] == 2020) & (tasas_pd['Country_name'] == 'Australia'))]['value']))

        if(data['country'][i]=="CA"):
            data_sale_price_us= (float(data['app_sale_price'][i])/float(tasas_pd.loc[((tasas_pd['currency_year'] == 2020) & (tasas_pd['Country_name'] == 'Canada'))]['value']))

        if(data['country'][i]=="BR"):
            data_sale_price_us= (float(data['app_sale_price'][i])/float(tasas_pd.loc[((tasas_pd['currency_year'] == 2020) & (tasas_pd['Country_name'] == 'Brazil'))]['value']))

        if(data['country'][i]=="FR" or data['country'][i]=="DE" or data['country'][i]=="IT" or data['country'][i]=="NL" or data['country'][i]=="ES" ):
            data_sale_price_us= (float(data['app_sale_price'][i])/float(tasas_pd.loc[((tasas_pd['currency_year'] == 2020) & (tasas_pd['Country_name'] == 'Germany') & (tasas_pd['currency'] == 'EUR'))]['value']))

        if(data['country'][i]=="IN"):
            data_sale_price_us= (float(data['app_sale_price'][i])/float(tasas_pd.loc[((tasas_pd['currency_year'] == 2020) & (tasas_pd['Country_name'] == 'India'))]['value']))

        if(data['country'][i]=="TR"):
            data_sale_price_us= (float(data['app_sale_price'][i])/float(tasas_pd.loc[((tasas_pd['currency_year'] == 2020) & (tasas_pd['Country_name'] == 'Turkey'))]['value']))

        if(data['country'][i]=="AE"):
            data_sale_price_us= (float(data['app_sale_price'][i])/0.27)

        if(data['country'][i]=="GB"):
            data_sale_price_us= (float(data['app_sale_price'][i])/float(tasas_pd.loc[((tasas_pd['currency_year'] == 2020) & (tasas_pd['Country_name'] == 'United Kingdom'))]['value']))

        if(data['country'][i]=="JP"):
            data_sale_price_us= (float(data['app_sale_price'][i])/float(tasas_pd.loc[((tasas_pd['currency_year'] == 2020) & (tasas_pd['Country_name'] == 'Japan'))]['value']))

        if(data['country'][i]=="SG"):
            data_sale_price_us= (float(data['app_sale_price'][i])*0.74)
            
        products_standard_price['isbestseller'].append(data['isbestseller'][i])
        products_standard_price['product_id'].append(data['product_id'][i])
        products_standard_price['app_sale_price'].append(data['app_sale_price'][i])
        products_standard_price['app_sale_price_currency'].append(data['app_sale_price_currency'][i])
        products_standard_price['isprime'].append(data['isprime'][i])
        products_standard_price['evaluate_rate'].append(data['evaluate_rate'][i])
        products_standard_price['country'].append(data['country'][i])
        products_standard_price['app_sale_price_us'].append(data_sale_price_us)

    return products_standard_price



In [110]:
external_products_standard_price = pd.DataFrame(generate_app_price_us(external_products_2,tasas_pd))

In [111]:
external_products_standard_price_spark=spark.createDataFrame(external_products_standard_price) 

In [112]:
external_products_standard_price_spark_2=external_products_standard_price_spark.withColumn("app_sale_price",external_products_standard_price_spark.app_sale_price.cast("decimal(10,2)")) \
    .withColumn("app_sale_price_us",external_products_standard_price_spark.app_sale_price_us.cast("decimal(10,2)"))

In [114]:
external_products_standard_price_spark_2.write \
  .format("bigquery") \
  .option("table","becade_rgarciaf.external_products_standard_price") \
  .option("temporaryGcsBucket", "amazon_bucket_ramiro") \
  .mode('overwrite') \
  .save()

In [ ]:
################################ 2 ##############################

In [115]:
from pyspark.sql.functions import sum, col, desc, avg, count, countDistinct

In [116]:
external_products_avg_price = external_products_standard_price_spark_2.groupBy("product_id").agg(avg("app_sale_price_us").alias("avg_price_us"),countDistinct("country").alias("count_country")) 

In [117]:
external_products_avg_price.write \
  .format("bigquery") \
  .option("table","becade_rgarciaf.external_products_avg_price") \
  .option("temporaryGcsBucket", "amazon_bucket_ramiro") \
  .mode('overwrite') \
  .save()

In [118]:
########################## 3 #########################

In [119]:
external_products_id=external_products_standard_price_spark_2.select("product_id").distinct()

In [120]:
external_products_standard_price_spark_2=external_products_standard_price_spark.withColumn("app_sale_price",external_products_standard_price_spark.app_sale_price.cast("float")) \
    .withColumn("app_sale_price_us",external_products_standard_price_spark.app_sale_price_us.cast("float"))

In [121]:
external_products_id_pd=external_products_id.toPandas()
external_products_standard_price_pd = external_products_standard_price_spark_2.toPandas()

In [122]:
def find_max_and_min(data,product_id):
    print("arrancando con todaaaaa")
    products_price_ranges = {'product_id': [] ,'max_price': [], 'max_country': [],'min_price' : [],'min_country':[]}  
    max_country=''
    max_price=0
    min_price=10000000000
    min_country=''
    j=0
    for product in product_id['product_id']:
        if(j%100==0):
            print(j)
        for i in data.index:
            if (data["product_id"][i]==product):
                if max_price < data["app_sale_price_us"][i]:
                    max_price=data["app_sale_price_us"][i]
                    max_country = data["country"][i]
                if min_price > data["app_sale_price_us"][i]:
                    min_price=data["app_sale_price_us"][i]
                    min_country = data["country"][i]
        products_price_ranges['product_id'].append(product)
        products_price_ranges['max_price'].append(max_price)
        products_price_ranges['max_country'].append(max_country)
        products_price_ranges['min_price'].append(min_price)
        products_price_ranges['min_country'].append(min_country)
        j=j+1
        max_price=0
        max_country=''
        min_price=10000000000
        min_country=''
    return products_price_ranges

In [123]:
external_products_standard_price = find_max_and_min(external_products_standard_price_pd,external_products_id_pd)

arrancando con todaaaaa
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600


In [124]:
external_products_price_range_pd=pd.DataFrame(external_products_standard_price)

In [125]:
external_products_price_ranges_spark=spark.createDataFrame(external_products_price_range_pd) 


In [126]:
external_products_price_ranges_spark_2=external_products_price_ranges_spark.withColumn("max_price",external_products_price_ranges_spark.max_price.cast("decimal(10,2)")) \
    .withColumn("min_price",external_products_price_ranges_spark.min_price.cast("decimal(10,2)"))

In [127]:
external_products_price_ranges_spark_2.write \
  .format("bigquery") \
  .option("table","becade_rgarciaf.external_products_price_range") \
  .option("temporaryGcsBucket", "amazon_bucket_ramiro") \
  .mode('overwrite') \
  .save()

In [ ]:
####################################################### 4 #######################################################

In [128]:
table_standar_price = "becade_rgarciaf.external_products_standard_price "
external_standar_price = spark.read \
  .format("bigquery") \
  .option("table", table_standar_price) \
  .load()

In [129]:
external_standar_price_pd=external_standar_price.toPandas()

In [130]:
def filter_evaluate_range(data):
    for i in data.index:
        if(data["evaluate_rate"][i]=="" or data["evaluate_rate"][i]==None):
            data["evaluate_rate"][i]="0.0"
        data["evaluate_rate"][i] = ''.join((filter(lambda x: x in '0123456789,.', data["evaluate_rate"][i])))
        if(data["evaluate_rate"][i].count('5') >1 ):
            if( data["evaluate_rate"][i][data["evaluate_rate"][i].rfind('5')-1] == ',' or '.' ) :
                list1= list(data["evaluate_rate"][i])
                list1[data["evaluate_rate"][i].find('5')]=""
                data["evaluate_rate"][i]=''.join(list1)
            if(len(data["evaluate_rate"][i]) > (data["evaluate_rate"][i].find('5')+1) ):	
                if (data["evaluate_rate"][i][data["evaluate_rate"][i].find('5')+1] ==  ',' or '.' ) :
                    list1 = list(data["evaluate_rate"][i])
                    list1[data["evaluate_rate"][i].rfind('5')]=""
                    data["evaluate_rate"][i]=''.join(list1)
        else:
            data["evaluate_rate"][i] = ''.join((filter(lambda x: x not in '5', data["evaluate_rate"][i])))
        data["evaluate_rate"][i]=data["evaluate_rate"][i].replace(",",".")

    return data

In [131]:
external_standar_price_pd_filtered= filter_evaluate_range(external_standar_price_pd)

/opt/conda/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/anaconda/lib/py

In [132]:
external_standar_price_spark_filtered=spark.createDataFrame(external_standar_price_pd_filtered) 

In [133]:
from pyspark.sql.functions import sum, col, desc, avg, count

In [134]:
external_product_rate_avg=external_standar_price_spark_filtered.groupBy("product_id")\
    .agg(avg("evaluate_rate").alias("avg_evaluation_rate"),count("country").alias("country_count"))

In [135]:
external_product_rate_avg_2 = external_product_rate_avg.withColumn("avg_evaluation_rate",external_product_rate_avg.avg_evaluation_rate.cast("decimal(10,2)"))

In [136]:
external_product_rate_avg_2.write \
  .format("bigquery") \
  .option("table","becade_rgarciaf.external_product_rate_avg") \
  .option("temporaryGcsBucket", "amazon_bucket_ramiro") \
  .mode('overwrite') \
  .save()